Features to add:
- Change appearance mode button (light/dark)
- Video frame slider (across the bottom of the frame)
- Options panel (right side) **WIP**
- Video filepath (read)
- Exporting (+ filepath)
- Frame skip rate
- User-adjusted image size
- Refresh image after the parameters have been adjusted (without skipping to next frame)

In [9]:
import tkinter
import tkinter.messagebox
import customtkinter
import cv2
from PIL import Image, ImageTk
import time
from screeninfo import get_monitors
import numpy as np

from src.imageProcessing import CannyLines, MixedCannyLines, blurImages

customtkinter.set_appearance_mode("System")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("blue")  # Themes: "blue" (standard), "green", "dark-blue"


class GUI(customtkinter.CTk):
    def __init__(self):
        super().__init__()
        # Images settings
        self.filePath = 'Input Videos\Arsenal_goal.mp4'
        self.indexer = 0
        self.cap = cv2.VideoCapture(self.filePath)
        self.length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        self.imgParams = {'length' : int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT)),
                        'current frame' : 0,
                        'framerate' : 1,
                        'playing' : False, 
                        'blurring' : False, 
                            'blur coeff' : 0.2,
                        'multiLine' : False,
                            'lower bound' : 0, 
                            'upper bound' : 60, 
                            'dilation' : 1,
                            'ranges' : [0,100,200,255], 
                            'dilations' : [1,2,4],
                        'fadeout' : False}


        # get first frame
        ret, frame = self.cap.read()
        self.indexer += self.imgParams['framerate']
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (720,480))
        self.currentImage = frame
        self.prevImage =  np.zeros_like(frame, dtype=np.uint8)


        # configure window
        self.title("Graphic design is my passion.py")
        monitors = get_monitors()
        for m in monitors:
            if m.is_primary == True:
                width = m.width, 
                height = m.height

        self.geometry(f"{1300}x{600}")

        # configure grid layout (4x4)
        self.grid_columnconfigure((0, 2), weight=0)
        self.grid_columnconfigure((1), weight=2)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        # SIDEBAR (column 0, all rows)
        self.sidebar_frame = customtkinter.CTkFrame(self, width=250, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.make_left_sidebar()

        
        # IMAGE FRAME
        self.image_frame = customtkinter.CTkFrame(self)
        self.image_frame.grid(row=0, column=1, rowspan=4, columnspan=1, padx=(10, 10), pady=(40, 40), sticky="nsew")

        # TIME SLIDER
        self.time_slider = customtkinter.CTkSlider(self.image_frame, from_=0, to=self.length, number_of_steps=self.length, progress_color="#36719F")
        self.time_slider.grid(row=5, column=0, rowspan=3, padx=(10, 10), pady=(10, 20), sticky="ew")
        self.time_slider.bind("<ButtonRelease-1>", self.slider_func)

        imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame))    
        self.panel = tkinter.Label(master=self.image_frame, width=720, height=480)
        self.panel.grid(row=1,column=0,rowspan=3,columnspan=1)
        self.next_frame()


        # RIGHT-SIDE PANEL (tabs)
        self.tabview = customtkinter.CTkTabview(self, width=300)
        self.tabview.grid(row=0, column=2, rowspan=4, padx=(10, 10), pady=(0, 10), sticky="nsew")
        
        self.tabview.add("Edges"), self.tabview.add("Blurring"), self.tabview.add("Fadeout")
        # EDGES TAB
        self.EdgeVis = None
        self.edges_tab_appearance()
        # BLURRING TAB
        self.labelblur = customtkinter.CTkLabel(master=self.tabview.tab("Blurring"), text="% of Previous Frame to Overlay:")
        self.labelblur.grid(row=0, column=0)

        self.blur_coeff_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Blurring"), textvariable=tkinter.StringVar(value=self.imgParams['blur coeff']), width=40)
        self.blur_coeff_txtbx.grid(row=1, column=0)
        self.blur_coeff_txtbx.bind("<KeyRelease>", self.blur_params)

        self.blur_slider = customtkinter.CTkSlider(master=self.tabview.tab("Blurring"), from_=0, to=1, number_of_steps=10)
        self.blur_slider.grid(row=3, column=0, padx=(20, 10), pady=(10, 10), sticky="ew")
        self.blur_slider.bind("<ButtonRelease-1>", self.blur_params)

    def slider_func(self, value):
        val = int(self.time_slider.get())
        self.cap.set(cv2.CAP_PROP_POS_FRAMES,val)
        self.indexer = val
        self.next_frame()

    def make_left_sidebar(self):
        # change file
        self.file_label = customtkinter.CTkLabel(master=self.sidebar_frame, text="Video Filepath:")
        self.file_label.grid(row=0, column=0)
        self.file_path_txtbx = customtkinter.CTkEntry(master=self.sidebar_frame, textvariable=tkinter.StringVar(value=self.filePath), width=220)
        self.file_path_txtbx.grid(row=1, column=0)
        self.update_file_button = customtkinter.CTkButton(master=self.sidebar_frame, text="Load File", fg_color="transparent", 
                                                                border_width=2, text_color=("gray10", "#DCE4EE"), command=lambda: self.update_filepath())
        self.update_file_button.grid(row=2, column=0, pady=10)

        text_var = tkinter.StringVar(value="Play Clip")
        self.play_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.play_switch_func())
        self.play_switch.grid(row=3, column=0, pady=30, padx=20, sticky="n")

        # change framerate
        self.framerate_label = customtkinter.CTkLabel(master=self.sidebar_frame, text="Skip Frames:")
        self.framerate_label.grid(row=4, column=0, columnspan=2)
        self.framerate_txtbx = customtkinter.CTkEntry(master=self.sidebar_frame, textvariable=tkinter.StringVar(value=self.imgParams['framerate']), width=40)
        self.framerate_txtbx.grid(row=5, column=0, pady=(0,20))
        self.framerate_txtbx.bind("<KeyRelease>", self.update_framerate)

        # change main settings
        text_var = tkinter.StringVar(value="MultiLine")
        self.multiLine_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.multiLine_switch.grid(row=6, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="Blurring")
        self.blurring_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.blurring_switch.grid(row=7, column=0, pady=10, padx=20, sticky="n")

        text_var = tkinter.StringVar(value="Fadeout")
        self.fadeout_switch = customtkinter.CTkSwitch(master=self.sidebar_frame, textvariable=text_var,command=lambda: self.left_panel_switches())
        self.fadeout_switch.grid(row=8, column=0, pady=10, padx=20, sticky="n")

    def blur_params(self, value):
        slider_val = round(float(self.blur_slider.get()), 1)
        txtbx_val = round(float(self.blur_coeff_txtbx.get()), 1)
        #round(number,1)

        if slider_val != self.imgParams['blur coeff']:
            self.imgParams['blur coeff'] = slider_val
            self.blur_coeff_txtbx.configure(textvariable=tkinter.StringVar(value=slider_val))
        elif txtbx_val != self.imgParams['blur coeff']:
            self.imgParams['blur coeff'] = txtbx_val
            self.blur_slider.set(textvariable=tkinter.StringVar(value=txtbx_val))


    def update_framerate(self, value):
        try:   self.imgParams['framerate'] = int(self.framerate_txtbx.get())
        except:   pass # in case the box is empty, or a letter is typed into it...
    
    def update_params(self):
        if self.imgParams['multiLine'] == True:
            self.imgParams['ranges'] = [int(self.edge_left_bound.get()), int(self.edge_middle_left_bound.get()), int(self.edge_middle_right_bound.get()), int(self.edge_right_bound.get())]
            self.imgParams['dilations'] = [int(self.edge_left_line.get()), int(self.edge_middle_line.get()), int(self.edge_right_line.get())]
        else:
            self.imgParams['lower bound'] = int(self.lower_bound_txtbx.get())
            self.imgParams['upper bound'] = int(self.upper_bound_txtbx.get())
            self.imgParams['dilation'] = int(self.line_dil_txtbx.get())
        

    def edges_tab_appearance(self):
        # change Edge tab to show options for multipline line thicknesses
        if self.imgParams['multiLine'] == True and self.EdgeVis != "multi": 
            if self.EdgeVis == 'single':
                self.edge_LB_label.grid_remove()
                self.lower_bound_txtbx.grid_remove()
                self.edge_UB_label.grid_remove()
                self.upper_bound_txtbx.grid_remove()
                self.edge_dil_label.grid_remove()
                self.line_dil_txtbx.grid_remove()
                self.update_button_edges.grid_remove()

            self.tabview.tab("Edges").grid_columnconfigure((0,1,2,3), weight=1)
            self.tabview.tab("Edges").grid_rowconfigure((0,1,2,3,4,5,6), weight=0)

            # Add stuff here!
            self.edge_bounds_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Bound Ranges:")
            self.edge_bounds_label.grid(row=1, column=1, columnspan=6)
            self.edge_left_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][0]))
            self.edge_left_bound.grid(row=2,column=0,pady=10)
            self.edge_middle_left_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][1]))
            self.edge_middle_left_bound.grid(row=2,column=2,pady=10)
            self.edge_middle_right_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][2]))
            self.edge_middle_right_bound.grid(row=2,column=4,pady=10)
            self.edge_right_bound = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['ranges'][3]))
            self.edge_right_bound.grid(row=2,column=6,pady=10)

            self.edge_lines_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Line Thicknesses:")
            self.edge_lines_label.grid(row=3, column=1, columnspan=6)
            self.edge_left_line = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['dilations'][0]))
            self.edge_left_line.grid(row=4,column=1,pady=10)
            self.edge_middle_line = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['dilations'][1]))
            self.edge_middle_line.grid(row=4,column=3,pady=10)
            self.edge_right_line = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), width=40, textvariable=tkinter.StringVar(value=self.imgParams['dilations'][2]))
            self.edge_right_line.grid(row=4,column=5,pady=10)

            self.update_button_edges = customtkinter.CTkButton(master=self.tabview.tab("Edges"), text="Update values", fg_color="transparent", 
                                                                border_width=2, text_color=("gray10", "#DCE4EE"), command=lambda: self.update_params())
            self.update_button_edges.grid(row=6, column=1, columnspan=6, pady=30)

            self.EdgeVis = "multi"

        # change Edge tab to show options for single line thickness
        elif self.imgParams['multiLine'] == False and self.EdgeVis != "single":
            if self.EdgeVis == 'multi':
                self.edge_bounds_label.grid_remove()
                self.edge_left_bound.grid_remove()
                self.edge_middle_left_bound.grid_remove()
                self.edge_middle_right_bound.grid_remove()
                self.edge_right_bound.grid_remove()

                self.edge_lines_label.grid_remove()
                self.edge_left_line.grid_remove()
                self.edge_middle_line.grid_remove()
                self.edge_right_line.grid_remove()

                self.update_button_edges.grid_remove()
            
            self.tabview.tab("Edges").grid_columnconfigure((0), weight=1)
            self.tabview.tab("Edges").grid_columnconfigure((1,2,3), weight=0)
            self.tabview.tab("Edges").grid_rowconfigure((0,1,2,3,4), weight=0)
            
            self.edge_LB_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Lower Bound:")
            self.edge_LB_label.grid(row=0, column=0)
            self.lower_bound_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=self.imgParams['lower bound']))
            self.lower_bound_txtbx.grid(row=1, column=0, pady=0, padx=10)
            self.edge_UB_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Upper Bound:")
            self.edge_UB_label.grid(row=2, column=0)
            self.upper_bound_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=self.imgParams['upper bound']))
            self.upper_bound_txtbx.grid(row=3, column=0, pady=0, padx=10)
            self.edge_dil_label = customtkinter.CTkLabel(master=self.tabview.tab("Edges"), text="Dilation:")
            self.edge_dil_label.grid(row=4, column=0)
            self.line_dil_txtbx = customtkinter.CTkEntry(master=self.tabview.tab("Edges"), textvariable=tkinter.StringVar(value=self.imgParams['dilation']))
            self.line_dil_txtbx.grid(row=5, column=0, pady=0, padx=10)
            self.update_button_edges = customtkinter.CTkButton(master=self.tabview.tab("Edges"), text="Update values", fg_color="transparent", 
                                                                border_width=2, text_color=("gray10", "#DCE4EE"), command=lambda: self.update_params())
            self.update_button_edges.grid(row=6, column=0, pady=30)

            self.EdgeVis = "single"

    def update_filepath(self):
        # change the filepath, close the current
        self.filePath = str(self.file_path_txtbx.get())
        self.cap.release()
        # load file, re-init the indexing and slider
        self.cap = cv2.VideoCapture(self.filePath)
        self.length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.indexer = 0
        self.time_slider.configure(to=self.length)
        self.time_slider.set(self.indexer)
        self.next_frame()

    # formulate a better while loop than this ? 
    def left_panel_switches(self):
        self.imgParams['multiLine'] = self.multiLine_switch.get()
        self.imgParams['blurring'] = self.blurring_switch.get()
        self.imgParams['fadeout'] = self.fadeout_switch.get()
        self.edges_tab_appearance()

    
    def play_switch_func(self):
        # get their values
        update = self.play_switch.get()
        if update: self.next_frame()
        

    def next_frame(self):
        # load the next image here
        if self.imgParams['framerate'] != 1:
            for i in range(self.imgParams['framerate']):
                ret, frame = self.cap.read()
                self.indexer += 1 # skips 5 frames forward
            #self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.indexer)
        
        ret, frame = self.cap.read()
        self.indexer += 1

        if ret:
            frame = cv2.resize(frame, (720,480))
            self.augmentImage(frame) 
            self.time_slider.set(int(self.indexer))
            self.update()  
            
            #time.sleep(0.05)       
            if self.play_switch.get(): self.next_frame() # call itself again, if we want that
            
        else:
            self.cap.release() # finished clip (restart it)
            self.cap = cv2.VideoCapture(self.filePath)
            self.indexer = 0
            if self.play_switch.get(): self.next_frame()


    def setFrame(self, frame): 
        imgtk = ImageTk.PhotoImage(image=Image.fromarray(frame))    
        self.panel.configure(image=imgtk)
        self.panel.image = imgtk

    def augmentImage(self, frame):
        if self.imgParams['multiLine'] == True:
            canny_lines = MixedCannyLines(frame, ranges=self.imgParams['ranges'], dilations=self.imgParams['dilations'])
        else:
            canny_lines = CannyLines(frame, lower_bound=self.imgParams['lower bound'],upper_bound=self.imgParams['upper bound'], dilation=self.imgParams['dilation'])

        if self.imgParams['blurring'] == True:
            result = blurImages(canny_lines, self.prevImage, blur=self.imgParams['blur coeff'])
        else:
            result = canny_lines

        self.setFrame(result)
        self.prevImage = result

    

root = GUI()
root.mainloop()

In [16]:
from tkinter import *
import threading
import time

class gui:
    def __init__(self, window):
        # play button
        self.play_frame = Frame(master=window, relief=FLAT, borderwidth=1)
        self.play_frame.grid(row=0, column=0, padx=1, pady=1)
        self.play_button = Button(self.play_frame, text="play", fg="blue", command=lambda: self.play(1))
        self.play_button.pack()
        # stop button
        self.stop_frame = Frame(master=window, relief=FLAT, borderwidth=1)
        self.stop_frame.grid(row=0, column=2, padx=1, pady=1)
        self.stop_button = Button(self.stop_frame, text="stop", fg="red", command=lambda: self.play(0))
        self.stop_button.pack()

    def play(self, switch):
        a.set(switch) # update 'a'
        print(a.get())

root = Tk()

a = IntVar(value=0)

def tester(trig):
    while True:
        value = trig.get()
        if value == 1:
            time.sleep(0.5)
            print ("running")
        elif value == 0:
            time.sleep(0.5)
            print ("not running")

t1 = threading.Thread (target = tester, args = [a], daemon = True)
t1.start()

app = gui(root)
root.mainloop()

not running
not running
not running
not running
not running
not running
not running
not running
not running
not running
0
not running
not running
not running
not running
not running
1
not running
running
running
running
running
running
running
running
running
running
running
running
running


running


Exception in thread Thread-3 (tester):
Traceback (most recent call last):
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_8148\84278952.py", line 28, in tester
  File "c:\Users\danie\miniconda3\envs\footballtracking\lib\tkinter\__init__.py", line 568, in get
    value = self._tk.globalgetvar(self._name)
RuntimeError: main thread is not in main loop
